In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import pysnooper
from typing import List

https://leetcode-cn.com/problemset/all/?search=898  LFU: 460  LRU: 146

https://space.bilibili.com/9880352/video?tid=36&page=9&keyword=&order=pubdate

In [3]:
set({1, 5, 67, 3, 0, 'aaa', 'a', 'b', 'aaa'})

{0, 1, 3, 5, 67, 'a', 'aaa', 'b'}

## 3. 460. LFU缓存
leetcode: [460. LFU缓存](https://leetcode-cn.com/problems/lfu-cache/): 设计并实现最不经常使用（LFU）缓存的数据结构。它应该支持以下操作：get 和 put。`get(key)` - 如果键存在于缓存中，则获取键的值（总是正数），否则返回 -1。`put(key, value)` - 如果键不存在，请设置或插入值。当缓存达到其容量时，它应该在插入新项目之前，使最不经常使用的项目无效。在此问题中，当存在平局（即两个或更多个键具有相同使用频率）时，最近最少使用的键将被去除。


思路：
结合前两道的思路：将最大频率栈中列表嵌套改成列表中嵌套双向链表。

细节上需要记录每个双向链表的尾部。


In [ ]:
class doubleLinkNode():
    def __init__(self, val):
        self.val = val
        self.prev = None
        self.next = None


class LFUCache:
    def __init__(self, capacity: int):
        self.cacheDic = {}
        self.capacity = capacity
        self.linkedLists = []
        self.minFreqs = []
        self.minFreqIndex = 0
        self.cnt = 0

    def get(self, key: int) -> int:
        if key in self.cacheDic:
            curNode = self.cacheDic[key]
            self.touch(curNode)
            return curNode.val[1]
        else:
            return -1
        

    def put(self, key: int, value: int) -> None:
        # 检查输入
        assert value >= 0, 'value must be a positive integer'
        
        if key not in self.cacheDic:
            self.cnt += 1
            listNode = doubleLinkNode([key, value, 1])
            if len(self.linkedLists) == 0:           
                self.linkedLists.append(listNode)
                # assign minFreq use
                self.minFreqIndex = 0
                self.minFreqs.append(listNode)
            else:
                listNode.next = self.linkedLists[0]
                self.linkedLists[0].prev = listNode
                self.linkedLists[0] = listNode
            self.cacheDic[key] = listNode
            # check length
            print(self.cnt)
            if self.cnt > self.capacity:
                # update minFreq use
                self.cnt -= 1
                self.cacheDic.pop(self.minFreqs[0].val[0])
                
                print(f'cnt: {self.cnt}, pop: {self.minFreq.val}')
                
                self.minFreq = self.minFreq.prev
                self.minFreq.next = None
        else:
            curNode = self.cacheDic[key]
            # update value
            curNode.val[1] = value
            self.touch(curNode)

    def touch(self, curNode):
        print(f'move to head: {curNode.val}, prev: {curNode.prev}, next: {curNode.next}')
        freq = curNode.val[2] + 1
        # update freq
        curNode.val[2] = freq
        # split it from orginal linkedList
        if curNode.prev:  # is not head
            print(f'{curNode.val} is not head')
            curNode.prev.next = curNode.next
            if curNode is self.minFreq:
                self.minFreq = curNode.prev
            if curNode.next:  # is not _tail
                curNode.next.prev = curNode.prev
        else:  # is _head
            self.linkedLists[freq - 2] = curNode.next
        curNode.prev, curNode.next = None, None
        # add to index = 'freq - 1'
        if len(self.linkedLists) < freq:
            self.linkedLists.append(curNode)
        elif self.linkedLists[freq - 1] is None:
            self.linkedLists[freq - 1] = curNode
        else:
            curNode.next = self.linkedLists[freq - 1]
            curNode.next.prev = curNode
            self.linkedLists[freq - 1] = curNode
        print(f'moved: {curNode.val}, prev: {curNode.prev}, next: {curNode.next}')
        


# Your LFUCache object will be instantiated and called as such:
cache = LFUCache(2)

cache.put(1, 1)
cache.put(2, 2)
assert cache.get(1) == 1
cache.put(3, 3)
assert cache.get(2) == -1
assert cache.get(3) == 3
cache.put(4, 4)
assert cache.get(1) == -1
assert cache.get(3) == 3
assert cache.get(4) == 4


## 2. 895. 最大频率栈
leetcode: [895. 最大频率栈](https://leetcode-cn.com/problems/maximum-frequency-stack/): 实现 FreqStack，模拟类似栈的数据结构的操作的一个类。FreqStack 有两个函数：
- `push(int x)`，将整数 x 推入栈中。
- `pop()`，它移除并返回栈中出现最频繁的元素。如果最频繁的元素不只一个，则移除并返回最接近栈顶的元素。


思路：  
创建一个列表和一个字典，列表中存储 N 个栈，字典存储数出现的频率。将出现次数为 m 的数压入列表索引为 m-1 的栈中。每次弹出时弹出列表中最后一个栈的元素。即：列表第 1 个位置的栈用来存储频率为 1 的数，列表第 2 个位置的栈用来存储频率为 2 的数，....

In [3]:
class FreqStack:
    def __init__(self):
        self.stacks = []
        self.eleFreq = {}

    def push(self, x: int) -> None:
        # 检查输入
        if not isinstance(x, int):
            return None

        self.eleFreq[x] = self.eleFreq.get(x, 0) + 1
        if len(self.stacks) < self.eleFreq[x]:
            self.stacks.append([x])
        else:
            self.stacks[self.eleFreq[x] - 1].append(x)

    def pop(self) -> int:
        mostFeqNum = self.stacks[len(self.stacks) - 1].pop()
        if not self.stacks[len(self.stacks) - 1]:
            self.stacks.pop()

        self.eleFreq[mostFeqNum] -= 1
        if self.eleFreq[mostFeqNum] == 0:
            self.eleFreq.pop(mostFeqNum)

        return mostFeqNum


# Your FreqStack object will be instantiated and called as such:
obj = FreqStack()
obj.push([])
obj.push(5)
obj.push(7)
obj.push(5)
obj.push(7)
obj.push(4)
obj.push(5)
assert obj.pop() == 5
assert obj.pop() == 7
assert obj.pop() == 5
assert obj.pop() == 4


## 1. 146. LRU缓存机制

leetcode: [146. LRU缓存机制](https://leetcode-cn.com/problems/lru-cache/): 运用你所掌握的数据结构，设计和实现一个  LRU (最近最少使用) 缓存机制。它应该支持以下操作： 获取数据 get 和 写入数据 put：

- 获取数据 `get(key)` - 如果密钥 (key) 存在于缓存中，则获取密钥的值（总是正数），否则返回 -1。
- 写入数据 `put(key, value)` - 如果密钥不存在，则写入其数据值。当缓存容量达到上限时，它应该在写入新数据之前删除最近最少使用的数据值，从而为新的数据值留出空间。


思路1：  
- 在 O(1) 时间内 get，故需要使用字典。
- 在 O(1) 时间内，删除尾部元素，可以使用列表或者链表
- 在 O(1) 时间内，将某元素移动到头部，需要使用双向链表。

在实现的细节上需要记录双向链表的头和尾节点，在双向链表初始化有两个思路：一种是**从空开始**，这样需要每次添加或者删除节点时判断是否为头或尾节点；另一种是**创建两个空的节点分别作为头和尾**，这样可以避免后续判断。

In [58]:
class doubleLinkNode():
    def __init__(self, val):
        self.val = val
        self.prev = None
        self.next = None


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cnt = 0
        self.cacheDic = {}
        self._head = None
        self._tail = None

    def get(self, key: int) -> int:
        if key in self.cacheDic:
            # get current node
            curNode = self.cacheDic.get(key)
            # move cur node to head
            self.moveNodeToHead(curNode)
            return curNode.val[1]
        else:
            return -1

    def put(self, key: int, value: int) -> None:
        assert value >= 0, 'value must be a positive integer'
        if key not in self.cacheDic:
            # add node to doubleLinkedList
            newNode = doubleLinkNode((key, value))
            if not self._head:
                self._head = newNode
                self._tail = newNode
            else:
                newNode.next = self._head
                self._head.prev = newNode
                self._head = newNode
            # add to cacheDic
            self.cacheDic[key] = newNode
            # check length
            self.cnt += 1
            if self.cnt > self.capacity:
                self.cnt -= 1
                self.cacheDic.pop(self._tail.val[0])
                self._tail = self._tail.prev
                self._tail.next = None
        else:
            # update value
            curNode = self.cacheDic.get(key)
            curNode.val = (key, value)
            # move current node to doubleLinkedlist head
            self.moveNodeToHead(curNode)

    def moveNodeToHead(self, curNode):
        if curNode != self._head:
            if curNode != self._tail:
                curNode.prev.next = curNode.next
                curNode.next.prev = curNode.prev
            else:
                # update tail node
                curNode.prev.next = None
                self._tail = curNode.prev
            # assign to head
            curNode.next = self._head
            self._head.prev = curNode
            self._head = curNode


# Your LRUCache object will be instantiated and called as such:
obj = LRUCache(capacity=1)
obj.put(2, 1)
assert obj.get(2) == 1
obj.put(3, 2)
assert obj.get(2) == -1
assert obj.get(3) == 2


思路2:  
使用 Python 的 `OrderedDict`, 其会记住并保持键插入的顺序(本质也是 hashtable + doubleLinkedList)。在 `get/put` 时取出当前的键值对，再重新插入，这样的字典的最后一个键就是最近的键。事实上 `OrderedDict` 就已经实现这方法 `move_to_end`。
- `popitem(last=True)`: last=False: 弹出第一对键值，即弹出最早插入的键值
- `pop(key)`: 弹出指定 key 的键值
- `move_to_end(key, last=True)`: last=True：移动 key 到最末尾，即最近使用的键

In [17]:
from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cnt = 0
        self.orderDic = OrderedDict()

    def get(self, key: int) -> int:
        if key in self.orderDic:
            '''
            value = self.orderDic.pop(key)
            self.orderDic[key] = value
            return value
            '''
            self.orderDic.move_to_end(key, last=True)
            return self.orderDic[key]
        else:
            return -1

    def put(self, key: int, value: int) -> None:
        assert value >= 0, 'value must be a positive integer'
        if key not in self.orderDic:
            self.cnt += 1
            self.orderDic[key] = value
            
            if self.cnt > self.capacity:
                self.cnt -= 1
                self.orderDic.popitem(last=False)
        else:
            # update value
            '''
            self.orderDic.pop(key)
            self.orderDic[key] = value
            '''
            self.orderDic.move_to_end(key, last=True)
            self.orderDic[key] = value

            
# Your LRUCache object will be instantiated and called as such:
obj = LRUCache(capacity=1)
obj.put(2, 1)
assert obj.get(2) == 1
obj.put(3, 2)
assert obj.get(2) == -1
assert obj.get(3) == 2